In [1]:
import numpy as np
from numpy import linalg as LA
np.random.seed(42)

In [2]:
#Setting up the lanczos algorithm
# Function to tri-diagonalize a matrix
def tridiag(a, b, c, k1=-1, k2=0, k3=1):
  return np.diag(a, k1) + np.diag(b, k2) + np.diag(c, k3)

# Lanczos algorithm
def lanczos(A, v1,m = 10):
  np.set_printoptions(precision=3, suppress=True)
  # First iteration steps
  x, y = [], []
  n = m
  v2, beta = 0.0, 0.0
  v1 = v1/np.linalg.norm(v1)
  Q = np.array(v1)

  for i in range(n):
    # Iteration steps
    
    w_prime = np.dot(A, v1)
    conj = np.matrix.conjugate(w_prime)
    alpha = np.dot(conj, v1)
    w = w_prime - alpha * v1 - beta * v2
    beta = np.linalg.norm(w)
    x.append(np.linalg.norm(alpha))
    

    # Reset
    if i < (n-1):
        y.append(beta)
    v2 = v1
    v1 = w/beta
    Q = np.column_stack((Q,v1))
  return Q, tridiag(y, x, y)


In [3]:
A = np.diag([3., 1., 2., 3., 4., 100.])
n = A.shape[1]
v_0 = np.zeros(n)
v_0.fill(1.)
v = v_0 / np.linalg.norm(v_0)

# Obtaining the tri-diagonal matrix T
Q, T = lanczos(A, v)
#print(f'Tridiagonalization of A: \n{T}')

# Finding the eigenvalues w and eigenvectors v of the tri-diagonal matrix
w, v = LA.eig(T)
#print(f'\nAssociated eigenvalues: \n{w}')
#print(f'\nAssociated eigenvectors: \n{v}')
print(T.shape)
print(Q.shape)


(10, 10)
(6, 11)


In [4]:
from minresScipy import *


The objective of MINRES is to solve Ax = b by minimsing residuals, for a symmetric A. We observe what happens in almost symmetric matrices by doing the following - 
1. Perform MINRES on a diagonal A
2. Perform MINRES on a diagonal A plus disturbances to make it non symmetric.
3. Perform MINRES on a symmetric non-diagonal A
4. Perform MINRES on a symmetric non-diagonal A plus disturbances to make it non symmetric.

In [5]:
'''
msg = [' beta2 = 0.  If M = I, b and x are eigenvectors    ',   # -1
            ' beta1 = 0.  The exact solution is x0          ',   # 0
            ' A solution to Ax = b was found, given rtol        ',   # 1
            ' A least-squares solution was found, given rtol    ',   # 2
            ' Reasonable accuracy achieved, given eps           ',   # 3
            ' x has converged to an eigenvector                 ',   # 4
            ' acond has exceeded 0.1/eps                        ',   # 5
            ' The iteration limit was reached                   ',   # 6
            ' A  does not define a symmetric matrix             ',   # 7
            ' M  does not define a symmetric matrix             ',   # 8
            ' M  does not define a pos-def preconditioner       ']   # 9

'''

"\nmsg = [' beta2 = 0.  If M = I, b and x are eigenvectors    ',   # -1\n            ' beta1 = 0.  The exact solution is x0          ',   # 0\n            ' A solution to Ax = b was found, given rtol        ',   # 1\n            ' A least-squares solution was found, given rtol    ',   # 2\n            ' Reasonable accuracy achieved, given eps           ',   # 3\n            ' x has converged to an eigenvector                 ',   # 4\n            ' acond has exceeded 0.1/eps                        ',   # 5\n            ' The iteration limit was reached                   ',   # 6\n            ' A  does not define a symmetric matrix             ',   # 7\n            ' M  does not define a symmetric matrix             ',   # 8\n            ' M  does not define a pos-def preconditioner       ']   # 9\n\n"

Diagonal

In [6]:
#1. Diagonal A

A = np.diag([3., 1., 2., 3., 4., 100.])
# del_A = np.random.rand(6,6)
# A = A + 0.01*del_A
eigvals, eigvecs = LA.eig(A)
b = eigvecs[2]#np.array([2, 4, -1,5,6,7], dtype=float)

x, exitCode = minres(A, b)

print("x")
print(x)
print("b")
print(b)
print(exitCode)            # 0 indicates successful convergence

x
[0.  0.  0.5 0.  0.  0. ]
b
[0. 0. 1. 0. 0. 0.]
0


In [7]:
#2. Perturbed Diagonal A

b = eigvecs[2]#np.array([2, 4, -1,5,6,7], dtype=float)
del_A = np.random.rand(6,6)
for eta in [0, 0.001, 0.01, 0.02, 0.03, 0.1, 0.2]:
    A = np.diag([0., 1., 2., 3., 4., 100.]) + eta*del_A
    x, exitCode = minres(A, b)

    print(f"x for eta = {eta}")
    print(x)

    print("ExitCode :",exitCode)

x for eta = 0
[0.  0.  0.5 0.  0.  0. ]
ExitCode : 0
x for eta = 0.001
[-0.  -0.   0.5 -0.  -0.  -0. ]
ExitCode : 0
x for eta = 0.01
[-0.66  -0.003  0.502 -0.     0.     0.   ]
ExitCode : 0
x for eta = 0.02
[-0.674 -0.005  0.505 -0.     0.001  0.   ]
ExitCode : 0
x for eta = 0.03
[-0.787 -0.007  0.508  0.001  0.002  0.   ]
ExitCode : 0
x for eta = 0.1
[-1.013 -0.024  0.537  0.004  0.009  0.001]
ExitCode : 0
x for eta = 0.2
[-0.978 -0.049  0.571  0.005  0.018  0.001]
ExitCode : 30


In [8]:
print("The matrix A is")
print(np.diag([3., 1., 2., 3., 4., 100.]))
print("The matrix del_A is")
print(del_A)

The matrix A is
[[  3.   0.   0.   0.   0.   0.]
 [  0.   1.   0.   0.   0.   0.]
 [  0.   0.   2.   0.   0.   0.]
 [  0.   0.   0.   3.   0.   0.]
 [  0.   0.   0.   0.   4.   0.]
 [  0.   0.   0.   0.   0. 100.]]
The matrix del_A is
[[0.375 0.951 0.732 0.599 0.156 0.156]
 [0.058 0.866 0.601 0.708 0.021 0.97 ]
 [0.832 0.212 0.182 0.183 0.304 0.525]
 [0.432 0.291 0.612 0.139 0.292 0.366]
 [0.456 0.785 0.2   0.514 0.592 0.046]
 [0.608 0.171 0.065 0.949 0.966 0.808]]


Non - Diagonal

In [9]:
#1. Symmetric A
N = 6 #Size of the matrix
m = np.random.random_integers(-100,100,size=(N,N))
A= (m + m.T)/2
eigvals, eigvecs = LA.eig(A)
b = eigvecs[2]#np.array([2, 4, -1,5,6,7], dtype=float)
x, exitCode = minres(A, b)
print(A.shape)
print(b.shape)

print("x")
print(x)

print(exitCode) 

(6, 6)
(6,)
x
[-0.004 -0.007 -0.007  0.004 -0.005  0.003]
0


C:\Users\Adity\AppData\Local\Temp\ipykernel_55212\3179645864.py:3: DeprecationWarning: This function is deprecated. Please call randint(-100, 100 + 1) instead
  m = np.random.random_integers(-100,100,size=(N,N))


In [10]:
#2. Perturbed Symmetric A
b = eigvecs[2]#np.array([2, 4, -1,5,6,7], dtype=float)
print("b")
print(b)
m = np.random.random_integers(-100,100,size=(N,N))
del_A = np.random.rand(6,6)
for eta in [0, 0.001, 0.01, 0.02, 0.03, 0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,10,20]:
    A= (m + m.T)/2 + eta*del_A
    x, exitCode = minres(A, b)

    print(f"x for eta = {eta}")
    print(x)

    print("ExitCode :",exitCode)



b
[ 0.363 -0.468 -0.654  0.322 -0.339  0.06 ]
x for eta = 0
[0.004 0.003 0.003 0.005 0.    0.008]
ExitCode : 0
x for eta = 0.001
[0.004 0.003 0.003 0.005 0.    0.008]
ExitCode : 0
x for eta = 0.01
[0.004 0.003 0.003 0.005 0.    0.008]
ExitCode : 0
x for eta = 0.02
[0.004 0.003 0.003 0.005 0.    0.008]
ExitCode : 0
x for eta = 0.03
[0.004 0.003 0.003 0.005 0.    0.008]
ExitCode : 0
x for eta = 0.1
[0.004 0.003 0.003 0.005 0.    0.008]
ExitCode : 0
x for eta = 0.2
[0.004 0.003 0.003 0.005 0.    0.008]
ExitCode : 0
x for eta = 0.3
[0.004 0.003 0.003 0.005 0.    0.008]
ExitCode : 0
x for eta = 0.4
[0.004 0.003 0.003 0.005 0.    0.008]
ExitCode : 0
x for eta = 0.5
[0.004 0.003 0.003 0.005 0.    0.008]
ExitCode : 0
x for eta = 0.6
[0.004 0.003 0.003 0.005 0.    0.008]
ExitCode : 0
x for eta = 0.7
[0.004 0.003 0.003 0.005 0.    0.008]
ExitCode : 0
x for eta = 0.8
[0.004 0.003 0.003 0.005 0.    0.008]
ExitCode : 0
x for eta = 10
[0.003 0.006 0.005 0.006 0.002 0.007]
ExitCode : 0
x for eta = 20

C:\Users\Adity\AppData\Local\Temp\ipykernel_55212\1158544459.py:5: DeprecationWarning: This function is deprecated. Please call randint(-100, 100 + 1) instead
  m = np.random.random_integers(-100,100,size=(N,N))


In [11]:
print("The matrix A is")
print((m+m.T)/2)
print("The matrix del_A is")
print(del_A)

The matrix A is
[[-19.   16.5 -30.  -57.5  -7.5  93.5]
 [ 16.5 -60.   13.  -24.  -17.  -32.5]
 [-30.   13.  -92.  -16.5 -34.  -27. ]
 [-57.5 -24.  -16.5  35.    1.   61.5]
 [ -7.5 -17.  -34.    1.  -73.  -17. ]
 [ 93.5 -32.5 -27.   61.5 -17.  -53. ]]
The matrix del_A is
[[0.371 0.669 0.666 0.591 0.275 0.561]
 [0.383 0.972 0.849 0.722 0.236 0.256]
 [0.04  0.711 0.111 0.439 0.202 0.896]
 [0.475 0.563 0.696 0.139 0.604 0.54 ]
 [0.203 0.943 0.599 0.695 0.88  0.624]
 [0.296 0.105 0.457 0.218 0.417 0.883]]
